In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import os 
os.listdir('.')

['car-price-data.csv', 'homework.md', 'homework3.ipynb', 'notebook.ipynb']

In [48]:
data = 'car-price-data.csv'
df = pd.read_csv(data)
df.rename(columns={'MSRP':"price"},inplace=True)
filtred_columns=['Make',
'Model',
'Year',
'Engine HP',
'Engine Cylinders',
'Transmission Type',
'Vehicle Style',
'highway MPG',
'city mpg',
'price']
df=df[filtred_columns]
df.head()

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


In [51]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
numerical_columns = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')
df.fillna(0, inplace=True)
for i in df.columns:
    print(df[i].unique()[:5])
    print(df[i].nunique())
    print()

['bmw' 'audi' 'fiat' 'mercedes-benz' 'chrysler']
48

['1_series_m' '1_series' '100' '124_spider' '190-class']
914

[2011 2012 2013 1992 1993]
28

[335. 300. 230. 320. 172.]
357

[ 6.  4.  5.  8. 12.]
9

['manual' 'automatic' 'automated_manual' 'direct_drive' 'unknown']
5

['coupe' 'convertible' 'sedan' 'wagon' '4dr_hatchback']
16

[26 28 27 25 24]
59

[19 20 18 17 16]
69

[46135 40650 36350 29450 34500]
6049



In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  int64  
 3   engine_hp          11914 non-null  float64
 4   engine_cylinders   11914 non-null  float64
 5   transmission_type  11914 non-null  object 
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
 9   price              11914 non-null  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 930.9+ KB


# Question 1

In [53]:
df['transmission_type'].mode()
# automatic

0    automatic
Name: transmission_type, dtype: object

# Question 2

In [55]:
df.select_dtypes(include='number').corr()
# highway_mpg and city_mpg

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [65]:
mean_price=df['price'].mean()
df['above_average']=df['price'].apply(lambda x: 1 if x>mean_price else 0)


In [71]:
Train=df.copy()

In [72]:
df_full_train, df_test = train_test_split(Train, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values



print(len(df_train), len(df_val), len(df_test))

7148 2383 2383


# Question 3

In [73]:

from sklearn.metrics import mutual_info_score
def mutual_info_score_aa(series):
    return round(mutual_info_score(series, df_train['above_average']),2)

mi = df_train[categorical_columns].apply(mutual_info_score_aa)
mi.sort_values(ascending=False)

# transmission_type

model                0.47
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [74]:
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']


In [75]:
from sklearn.feature_extraction import DictVectorizer
dv=DictVectorizer(sparse=False)
train_dict=df_train[categorical_columns +numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)


In [76]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [77]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
y_pred = model.predict_proba(X_val)[:, 1] # return only that propbs that predict class 1
decision=(y_pred>=0.5)



# Question 4

In [80]:
round((y_val==decision).mean(),2)
# 0. 95

0.94

In [83]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = decision.astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred.prediction == df_pred.actual


In [84]:
original_metric=df_pred.correct.mean()
metrics={'original':original_metric}


0.9353755770037767

In [86]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'city_mpg': 0.062,
 'engine_cylinders': -0.146,
 'engine_hp': 0.039,
 'highway_mpg': 0.015,
 'make=acura': 0.866,
 'make=alfa_romeo': 0.965,
 'make=aston_martin': 0.455,
 'make=audi': 3.013,
 'make=bentley': 0.124,
 'make=bmw': 2.319,
 'make=bugatti': 0.0,
 'make=buick': -0.614,
 'make=cadillac': 2.587,
 'make=chevrolet': -1.708,
 'make=chrysler': -0.957,
 'make=dodge': -2.989,
 'make=ferrari': 0.575,
 'make=fiat': -0.308,
 'make=ford': -2.003,
 'make=genesis': 0.227,
 'make=gmc': -0.884,
 'make=honda': -1.389,
 'make=hummer': -0.006,
 'make=hyundai': -2.697,
 'make=infiniti': 0.215,
 'make=kia': -1.716,
 'make=lamborghini': 0.005,
 'make=land_rover': 2.145,
 'make=lexus': 1.553,
 'make=lincoln': 1.454,
 'make=lotus': 3.73,
 'make=maserati': 0.978,
 'make=maybach': 0.002,
 'make=mazda': -1.512,
 'make=mclaren': 0.0,
 'make=mercedes-benz': 0.663,
 'make=mitsubishi': -1.532,
 'make=nissan': -1.353,
 'make=oldsmobile': -1.162,
 'make=plymouth': -0.133,
 'make=pontiac': -2.839,
 'make=por

In [87]:
def run(small):
    dicts_train_small = df_train[small].to_dict(orient='records')
    dicts_val_small = df_val[small].to_dict(orient='records')
    dv_small = DictVectorizer(sparse=False)
    dv_small.fit(dicts_train_small)
    X_train_small = dv_small.transform(dicts_train_small)
    model_small  = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model_small.fit(X_train_small, y_train)
    X_val = dv_small.transform(val_dict)
    y_pred = model_small.predict_proba(X_val)[:, 1] # return only that propbs that predict class 1
    decision=(y_pred>=0.5)
    acc=(y_val==decision).mean()
    return(acc)



In [88]:
to_filter=['year',
'engine_hp',
'transmission_type',
'city_mpg']


small=categorical_columns +numerical_columns
print(len(small))
print("original feature vector", small)
for i in range(len(to_filter)):
    small=categorical_columns +numerical_columns
    small.remove(to_filter[i])
    print(f"hide feature: {to_filter[i]}")
    print("new feature vector:", small)
    accuracy = run(small)
    print("accuracy", accuracy )
    metrics[to_filter[i]]=accuracy

    

9
original feature vector ['make', 'model', 'transmission_type', 'vehicle_style', 'year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']
hide feature: year
new feature vector: ['make', 'model', 'transmission_type', 'vehicle_style', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']
accuracy 0.9517415023080151
hide feature: engine_hp
new feature vector: ['make', 'model', 'transmission_type', 'vehicle_style', 'year', 'engine_cylinders', 'highway_mpg', 'city_mpg']
accuracy 0.9227864036928242
hide feature: transmission_type
new feature vector: ['make', 'model', 'vehicle_style', 'year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg']
accuracy 0.9357952161141418
hide feature: city_mpg
new feature vector: ['make', 'model', 'transmission_type', 'vehicle_style', 'year', 'engine_hp', 'engine_cylinders', 'highway_mpg']
accuracy 0.936634494334872


# Question 5

In [89]:
print("year", metrics['original']-metrics['year'])
print("engine_hp", metrics['original']-metrics['engine_hp'])
print("transmission_type", metrics['original']-metrics['transmission_type'])
print("city_mpg", metrics['original']-metrics['city_mpg'])
# transmission_type

year -0.016365925304238416
engine_hp 0.01258917331095255
transmission_type -0.00041963911036513313
city_mpg -0.0012589173310952884


In [90]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import OneHotEncoder

# X_train_num = df_train[numerical_columns].values

# scaler = StandardScaler()
# #scaler = MinMaxScaler()

# X_train_num = scaler.fit_transform(X_train_num)
# ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# X_train_cat = ohe.fit_transform(df_train[categorical_columns    ].values)
# X_train = np.column_stack([X_train_num, X_train_cat])

# X_val_num = df_val[numerical_columns].values
# X_val_num = scaler.transform(X_val_num)

# X_val_cat = ohe.transform(df_val[categorical_columns].values)

# X_val = np.column_stack([X_val_num, X_val_cat])


In [93]:

print(len(df_train), len(df_val), len(df_test))
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
y_test = np.log1p(y_test)
dv=DictVectorizer(sparse=True)

train_dict=df_train[categorical_columns +numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)


7148 2383 2383


# Question 6

In [96]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

for a in [0.0, 0.01, 0.1, 1, 10]:

    r_model =Ridge(solver='sag',random_state=42, alpha=a,)
    r_model.fit(X_train, y_train)
    y_pred=r_model.predict(X_val)
    rmse=mean_squared_error(y_pred=y_pred, y_true=y_val,squared=False)
    # print(',y_pred',y_pred[:5], 'y_val',y_val[:5])
    print(a, rmse)
    # break

c:\Users\Anatolii\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_ridge.py:729: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


0.0 0.7313710366306405


c:\Users\Anatolii\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_ridge.py:729: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


0.01 0.7313710574636403


c:\Users\Anatolii\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_ridge.py:729: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


0.1 0.7313712449518462


c:\Users\Anatolii\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_ridge.py:729: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


1 0.7313731197871051


c:\Users\Anatolii\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_ridge.py:729: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  warnings.warn(


10 0.7313918634329672


In [ ]:
0.731371036630
0.731371057463
0.731371244951
0.731373119787
0.731391863432